In [5]:
import torch.nn.utils.spectral_norm as spectral_norm
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import general_utils  # assuming general_utils.pickle_load is available
from collections import defaultdict
from PIL import Image

In [6]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [7]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from ssd_network import  *
from ssd_dataset import  *

# prepare data

In [9]:
csv_file = r"C:\Users\izaol\fridge_food\jedzenie\train\_annotations.csv"
images_dir = r"C:\Users\izaol\fridge_food\jedzenie\train"
formatted_data, label_to_id = process_annotations(csv_file, images_dir)

with open('formatted_data.pkl', 'wb') as f:
    pickle.dump(formatted_data, f)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\izaol\\fridge_food\\jedzenie\\train\\_annotations.csv'

In [ ]:
with open('formatted_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
print(data)

# config

In [ ]:
config = {}

config['base_conv_conv_layers'] = [1, 16, 32, 64] 
config['base_conv_input_size'] = 32
base_conv = BaseConv(config['base_conv_conv_layers'], 
                     config['base_conv_input_size'], chosen_fm=[-2, -1],
                     norm=nn.BatchNorm2d, act_fn=nn.ReLU(), spectral=False)
base_size = pretty_print_module_list(base_conv.module_list, torch.zeros([1,1,config['base_conv_input_size'], config['base_conv_input_size']]))

In [ ]:
config['aux_conv_conv_layers'] = [64, 64, 64]
config['aux_conv_input_size'] = 8
aux_conv = AuxConv(config['aux_conv_conv_layers'], 
                   config['aux_conv_input_size'], norm=nn.BatchNorm2d, act_fn=nn.ReLU(), spectral=False)
aux_size = pretty_print_module_list(aux_conv.module_list, torch.zeros(base_size[-1]))

In [ ]:
config['device'] = 'cpu'

#model config
config['n_classes'] = 50

config['fm_channels'] = [base_size[i][1] for i in base_conv.fm_id] + [aux_size[i][1] for i in aux_conv.fm_id]
config['fm_size'] = [base_size[i][-1] for i in base_conv.fm_id] + [aux_size[i][-1] for i in aux_conv.fm_id]
config['n_fm'] = len(config['fm_channels'])
config['fm_prior_aspect_ratio'] = [[1., 2., 3., 1/3, 0.5],
                         [1., 2., 3., 0.5, 1/3],
                         [1., 2., 3., 0.5, 1/3],
                         [1., 2., 3., 0.5, 1/3],
                         [1., 2., 0.5], 
                         [1., 2., 0.5]]

config['fm_prior_aspect_ratio'] = config['fm_prior_aspect_ratio'][:config['n_fm']]

config['fm_prior_scale'] = np.linspace(0.1, 0.9, config['n_fm']) #[0.2, 0.375, 0.55, 0.725, 0.9] # [0.1, 0.2, 0.375, 0.55, 0.725, 0.9] 
assert len(config['fm_prior_scale']) == len(config['fm_prior_aspect_ratio'])
config['n_prior_per_pixel'] = [len(i)+1 for i in config['fm_prior_aspect_ratio']] #in fm1, each pixel has 4 priors

# training config
config['checkpoint'] = ''#output_folder/'checkpoint.pth'
config['n_epoch'] = 40
config['save_frequency'] = 20
config['MultiStepLR_milestones'] = list(range(10, config['n_epoch'], 5))
config['gamma'] = 0.5
config['print_frequency'] = 1
config['batch_size'] = 20
config['iou_threshold'] = 0.5 #
config['lr'] = 1e-3
config['momentum'] = 0.9
config['weight_decay'] = 5e-4
config['pos_neg_ratio'] = 1.
config['clip_grad'] = None
config['multiboxloss_loc_cla_ratio'] = 0.25


utils.img_size = base_size[0][-1]

print(config)

In [ ]:
#create model
model = SSD(config, base_conv, aux_conv).to(config['device'])

In [ ]:
bias = []
not_bias = []
for name, param in model.named_parameters():
    if param.requires_grad:
        if 'bias' in name:
            bias.append(param)
        else:
            not_bias.append(param)

optimizer = torch.optim.Adam(
    [{'params': bias, 'lr': config['lr'] * 2}, {'params': not_bias}],
    lr=config['lr'],
    weight_decay=config['weight_decay']
)

scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer, 
    milestones=config['MultiStepLR_milestones'], 
    gamma=config['gamma']
)
print(model.priors_cxcy.shape)
criterion = MultiBoxLoss(priors_cxcy=model.priors_cxcy, config=config)
plot_loss = []

# load checkpoint
if config['checkpoint']:
    print('Loaded checkpoint')
    checkpoint = torch.load(config['checkpoint'], map_location=config['device'])
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    plot_loss = checkpoint['plot_loss']

current_lr = scheduler.get_last_lr()
print("Current learning rate:", current_lr)


# Dataset

In [ ]:
csv_file = r"C:\Users\izaol\fridge_food\jedzenie\train\_annotations.csv"
images_dir = r"C:\Users\izaol\fridge_food\jedzenie\train"
formatted_data, label_to_id = process_annotations(csv_file, images_dir)

with open('formatted_data.pkl', 'wb') as f:
    pickle.dump(formatted_data, f)

In [ ]:
numb_class = len(label_to_id)

In [ ]:
trainset = MyDataset('formatted_data.pkl', device=config['device'])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=config['batch_size'], collate_fn=trainset.collate_fn, shuffle=True)

In [ ]:
training_progress_bar = tqdm(range(config['n_epoch']))
epoch_progress_bar = tqdm(range(len(trainloader)))

for e in range(config['n_epoch']):
    training_progress_bar.update()
    epoch_progress_bar.reset()

    mean_loss = 0
    for img, boxes, labels in trainloader:
        epoch_progress_bar.update()
        
        img = img.unsqueeze(1)
        
        # img: tensor [n, c=1, h, w]
        # boxes: list[n] of tensor[n_object, 4]
        # labels: list[n] of tensor[n_object]
        loc_output, cla_output, _ = model(img)
        print("wymiar model: ", model.priors_cxcy.shape)
        print("wymiary loc: ", loc_output.shape, " wymiary cla: ", cla_output.shape, " box: ", len(boxes), " wymiary labels: ", len(labels))
        loss = criterion(loc_output, cla_output, boxes, labels)
        
        optimizer.zero_grad()
        loss.backward()

        if config['clip_grad']:
            utils.clip_gradient(optimizer, config['clip_grad'])
        
        optimizer.step()
        mean_loss += loss.item()
        # break
    scheduler.step()
    mean_loss /= len(trainloader)
    plot_loss.append(mean_loss)
    
    if (e%config['print_frequency'] == config['print_frequency'] - 1) or (e==0):
        print(mean_loss)
        print(criterion.loc_loss.item(), criterion.cla_loss.item())
